In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
print("Device: {}".format(device))

Device: cpu


In [ ]:
import numpy as np
import os
from imblearn.over_sampling import SMOTE 
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.tensorflow import balanced_batch_generator
from numpy.random import default_rng
rng = default_rng(seed = 31)
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
!pip install geopandas
!pip install libpysal
import geopandas as gpd
import shapely
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
import libpysal
from torch import nn, optim
import scipy.ndimage as ndimage
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from copy import deepcopy

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.3 MB 39.6 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 2.4 MB 5.0 MB/s 


In [ ]:
ss = StandardScaler()

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:
def load_data(year):
    os.getcwd()
    
    df = pd.DataFrame()
    path = "/content/drive/MyDrive/Uni/Tilburg/Thesis/" + str(year) + "/"
    for filename in os.listdir(path):
        df1 = pd.read_csv(path + filename)
        print(filename, df1.shape)
        if df1.geometry.isna().any():
            print(filename)
        df = pd.concat([df, df1])
    df = gpd.GeoDataFrame(df)
    # print(df.shape)
    df.geometry = df.geometry.apply(shapely.wkt.loads)
    
    df = df.reset_index()
    df = df.drop(["Unnamed: 0", "index"], axis = 1)
    return df

In [ ]:
# df15 = load_data(2015)
# df16 = load_data(2016)
df17 = load_data(2017)
df18 = load_data(2018)
df19 = load_data(2019)

's-Gravenhage17.csv_filled_.csv (9018, 45)
's-Gravenhage18.csv_filled_.csv (9111, 45)
's-Gravenhage19.csv_filled_.csv (9182, 45)


In [ ]:
X_train = df17[df17.columns[:-4]]
Y_train = df17.y
Y1_train = df17["y-1"]
# print(X_train)
# X_train = ss.fit_transform(X_train)

X_val = df18[df18.columns[:-4]]
Y_val = df18.y
Y1_val = df18["y-1"]

X_test = df19[df19.columns[:-4]]
Y_test = df19.y
Y1_test = df19["y-1"]


wt = libpysal.weights.DistanceBand.from_dataframe(df17, threshold=150, binary = True, silence_warnings = True)
wv =libpysal.weights.DistanceBand.from_dataframe(df18, threshold=150, binary = True, silence_warnings = True)

wtest =libpysal.weights.DistanceBand.from_dataframe(df19, threshold=150, binary = True, silence_warnings = True)

/usr/local/lib/python3.7/dist-packages/libpysal/weights/util.py:1073: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = np.vstack([np.array(shape.centroid) for shape in first_choice])


In [ ]:
neighbors = [wt.neighbors[x] for x in wt.neighbors]
Y1_train = np.array([Y1_train[x].sum() for x in neighbors])
Y1_train[np.where(Y1_train == 0)[0]] = 0.5

neighbors = [wv.neighbors[x] for x in wv.neighbors]
Y1_val = np.array([Y1_val[x].sum() for x in neighbors])
Y1_val[np.where(Y1_val == 0)[0]] = 0.5

neighbors = [wtest.neighbors[x] for x in wtest.neighbors]
Y1_test = np.array([Y1_test[x].sum() for x in neighbors])
Y1_test[np.where(Y1_test == 0)[0]] = 0.5

In [ ]:
X_train = ss.fit_transform(X_train)
X_train[:,-1] = Y1_train
X_val = ss.transform(X_val)
X_val[:,-1] = Y1_val
X_test = ss.transform(X_test)
X_test[:,-1] = Y1_test

In [ ]:
N_clusters = 3

cluster_model = KMeans(n_clusters = N_clusters, random_state = 1)
train_l = pd.DataFrame(cluster_model.fit_predict(X_train[:,:-1]))
val_l = pd.DataFrame(cluster_model.predict(X_val[:,:-1]))
test_l = pd.DataFrame(cluster_model.predict(X_test[:,:-1]))

In [ ]:
def get_batch(x, y, batch_size = 32):
    idxs = np.random.randint(len(x), size = batch_size)
    return x[idxs], y[idxs]

In [ ]:
class BigModel(nn.Module):
    def __init__(self,  drop_rate = 0.2):
        super(BigModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(39,100),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(100,350),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(350,500),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(500,250),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(250,1))

        self.final = nn.Sigmoid()
        self.name = "biggie"

    def forward(self, x):
        Y_1 = x[:,-1].reshape(-1,1) # take yt-1 of neighbors
        x = x[:,:-1]
        out = self.net(x) # get p
        
        out *= Y_1 # multiply with yt-1

        return out
       

In [ ]:
class MidModel(nn.Module):
    def __init__(self,  drop_rate = 0.2):
        super(MidModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(39,100),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(100,250),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(250,100),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(100,1)
             )

        self.final = nn.Sigmoid()
        self.name = "Malcom in the"

    def forward(self, x):
        Y_1 = x[:,-1].reshape(-1,1) # take yt-1 of neighbors
        x = x[:,:-1]
        out = self.net(x) # get p
        
        out *= Y_1 # multiply with yt-1

        return out
       

In [ ]:
class SmallModel(nn.Module):
    def __init__(self,  drop_rate = 0.2):
        super(SmallModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(39,50),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(50,50),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(50,1)
             )

        self.final = nn.Sigmoid()
        self.name = "It isn't about size"

    def forward(self, x):
        Y_1 = x[:,-1].reshape(-1,1) # take yt-1 of neighbors
        x = x[:,:-1]
        out = self.net(x) # get p
        
        out *= Y_1 # multiply with yt-1

        return out

In [ ]:
def train_model(X_train, Y_train, train_l, X_val, Y_val, val_l, num_epochs,lr, batch_per_e = 150):

    tot_auc = 0
    tot_model = np.NaN
    tot_param = 0
    


    print("train: {}, val: {}".format(Counter(train_l[0]), Counter(val_l[0])))

    auc_clust_hist = {}
    f1_clust_hist = {}
    clust_models = {}
    param_clust_hist = {} 

    for clust in train_l[0].unique():
      
      best_auc = 0
      best_f1 = 0
      best_model = np.NaN
        
      idxs = train_l[train_l[0]== clust].index
      xt = X_train[idxs]
      yt = Y_train[idxs]

      oversample = SMOTE()
      xt, yt = oversample.fit_resample(xt, yt)

      xt[xt[:,-1]>0.5][:,-1] =  np.ceil(xt[xt[:,-1] > 0.5][:,-1]) # 0.5 means no neighbors, otherwise make int
      xt = torch.tensor(xt).float().to(device)
      yt = torch.tensor(yt).float().to(device)

      
      idxs = val_l[val_l[0] == clust].index
      xv = X_val[idxs]
      yv = Y_val[idxs].values

      xv = torch.tensor(xv).float().to(device)
      yv = torch.tensor(yv).float().to(device)
      yn = yv.cpu().detach().numpy()

      for lr in [0.0000051, 0.00000051]:
        for size in ["big", "middle", "small"]:

            if size == "big":
              model = BigModel().to(device)
            elif size == "middle":
              model = MidModel().to(device)
            else: 
              model = SmallModel().to(device)

            optimizer = optim.Adam(model.parameters(), lr = lr)
            SigBCEloss = nn.BCEWithLogitsLoss()

            for epoch in range(num_epochs):
              epoch_loss = []
              model.train()
              for i in range(batch_per_e):

                optimizer.zero_grad()
                x,y = get_batch(xt, yt, 6)
                out = model(x) # get output from final linear layer

                loss = SigBCEloss(out, y.unsqueeze(1)) # put through sigmoid and calculate loss
                loss.backward()
                optimizer.step()
                      
              model.eval()
              
              out = model(xv)             

              yn = yv.cpu().detach().numpy()

              ROC = roc_auc_score(yn, model.final(out).detach())

              val_loss = SigBCEloss(out, yv.unsqueeze(1))
              out = out.cpu().squeeze().detach().numpy()
              
              
              preds = np.zeros(len(out))
              pos = out.argsort()[-((yv==1).sum()) : ]
              preds[pos] = 1

              acc = accuracy_score(yn, preds)
              f1 = f1_score(yn, preds)
              
              if ROC > best_auc:
                print("best UAC in cluster {}: {}".format(clust, ROC))
                best_auc = ROC
                best_f1 = f1
                param_clust_hist[clust] = [lr, size, epoch]
                clust_models[clust] = deepcopy(model.state_dict())

      # histories for this clust
      auc_clust_hist[clust] = best_auc
      f1_clust_hist[clust] = best_f1

    predictions = np.zeros(len(Y_val))
    for clust in train_l[0].unique():
      
      idxs = val_l[val_l[0] == clust].index
      xv = torch.tensor(X_val[idxs]).float().to(device)
      size = param_clust_hist[clust][1]
      if size == "big":
        model = BigModel().to(device)
        model.load_state_dict(clust_models[clust])
      elif size == "middle":
        model = MidModel().to(device)
        model.load_state_dict(clust_models[clust])
      else: 
        model = SmallModel().to(device)
        model.load_state_dict(clust_models[clust])

      preds = model(xv)

      preds = preds.cpu().squeeze().detach().numpy()
      predictions[idxs] = preds

    pos_idxs = predictions.argsort()[- (Y_val == 1).sum() : ]
    Yhat = np.zeros(len(Y_val))
    Yhat[pos_idxs] = 1
    auc = roc_auc_score(Y_val, predictions)
    f1 = f1_score(Y_val, Yhat)

    if auc > tot_auc:
      print("auc!! {}".format(auc))
      tot_auc = auc
      tot_model = clust_models
      tot_param = param_clust_hist

    return tot_model, tot_param, tot_auc


In [ ]:
n_epochs = 600
outs = []
# for lr in [0.1,0.01,0.001,0.0001]:
m, p, auc = train_model(X_train, Y_train, train_l, X_val, Y_val, val_l, n_epochs, 0.00000051)
  # outs.append(out1)

train: Counter({0: 4894, 1: 2882, 2: 1242}), val: Counter({0: 4897, 1: 2932, 2: 1282})
best UAC in cluster 2: 0.38906765676567656
best UAC in cluster 2: 0.540529231494578
best UAC in cluster 2: 0.5934229137199434
best UAC in cluster 2: 0.5935643564356438
best UAC in cluster 2: 0.5940240452616691
best UAC in cluster 2: 0.5948609146628949
best UAC in cluster 2: 0.5962753418198963
best UAC in cluster 2: 0.5967703913248469
best UAC in cluster 2: 0.5980551626591231
best UAC in cluster 2: 0.5991749174917491
best UAC in cluster 2: 0.5993752946723244
best UAC in cluster 2: 0.601119754832626
best UAC in cluster 2: 0.6013083451202264
best UAC in cluster 2: 0.6024516737388024
best UAC in cluster 2: 0.606034889203206
best UAC in cluster 2: 0.6088519566242339
best UAC in cluster 2: 0.6117220650636492
best UAC in cluster 2: 0.6140499764262141
best UAC in cluster 2: 0.6151933050447902
best UAC in cluster 2: 0.6167727487034418
best UAC in cluster 2: 0.6177746346063179
best UAC in cluster 2: 0.61804573

# TEST

In [ ]:
predictions = np.zeros(len(Y_test))
auc_clusters = {}
f1_clusters = {}
for clust in train_l[0].unique():
  
    idxs = test_l[test_l[0] == clust].index
    xt = torch.tensor(X_test[idxs]).float().to(device)
    yt = Y_test[idxs]

    size = p[clust][1]
    if size == "big":
        model = BigModel().to(device)
        model.load_state_dict(m[clust])
        model.eval()
    elif size == "middle":
        model = MidModel().to(device)
        model.load_state_dict(m[clust])
        model.eval()
    else: 
        model = SmallModel().to(device)
        model.load_state_dict(m[clust])
        model.eval()
    preds = model(xt)
    preds = preds.cpu().squeeze().detach().numpy()  
    auc = roc_auc_score(yt, preds)
    auc_clusters[clust] = auc

    Yhat = np.zeros(len(yt))
    pos_idxs = preds.argsort()[- (yt == 1).sum() : ]
    Yhat[pos_idxs] = 1
    f1_clusters[clust] = f1 = f1_score(yt, Yhat)

    predictions[idxs] = preds

pos_idxs = predictions.argsort()[- (Y_test == 1).sum() : ]
Yhat = np.zeros(len(Y_test))
Yhat[pos_idxs] = 1
auc = roc_auc_score(Y_test, predictions)
f1 = f1_score(Y_test, Yhat)


In [ ]:
p

{0: [5.1e-07, 'big', 599],
 1: [5.1e-06, 'big', 158],
 2: [5.1e-07, 'small', 315]}

In [ ]:
auc

0.6110609428855193

In [ ]:
f1

0.3696612665684831

In [ ]:
auc_clusters

{0: 0.667628706382697, 1: 0.6883599070798797}

In [ ]:
f1_clusters

{0: 0.2211981566820277, 1: 0.23870967741935487}

Beste: 3 clusters, middel, small. big. Allemaal 5.1e-06.

In [ ]:
plt.plot(out1[-3], alpha = 0.5)
plt.hlines(np.mean(out1[-3]), 0, n_epochs, color = "r")
plt.plot(out1[-1], alpha = 0.5, color = "green")

NameError: ignored

top auc seen = 0.709 

In [ ]:
print(np.max(out1[-4]))
plt.plot(out1[-4], alpha = 0.5)
plt.hlines(np.mean(out1[-4]), 0, n_epochs, color = "r")

In [ ]:
print(np.max(out1[-2]))
plt.plot(out1[-2], alpha = 0.5)
plt.hlines(np.mean(out1[-2]), 0, n_epochs, color = "r")

In [ ]:
loss = nn.BCEWithLogitsLoss()(b,y.unsqueeze(1))

In [ ]:
 # best auc: 0.675, f1: 0.476, lr: 0.00000051, epochs: 10000
class Model(nn.Module):
    def __init__(self,  drop_rate = 0.2):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(39,100),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(100,250),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(250,100),
            nn.ReLU(),
            nn.Linear(100,1)
             )

        self.final = nn.Sigmoid()

    def forward(self, x):
        Y_1 = x[:,-1].reshape(-1,1) # take yt-1 of neighbors
        x = x[:,:-1]
        out = self.net(x) # get p
        
        out *= Y_1 # multiply with yt-1

        return out, Y_1
       

In [ ]:
x = x.detach().numpy()
xv = X_val.detach().numpy()

In [ ]:
for i in range(x.shape[1] - 1):
  plt.hist(x[:,i], bins = 20, range = (-1,2), alpha = 0.5)
  plt.hist(xv[:,i], bins = 20, range = (-1,2), alpha = 0.5)
  plt.show()

In [ ]:
loss.backward()

In [ ]:
Oke, onderutssen even het een en ander typen

In [ ]:
"giab we alaaaaf!"